In [1]:
!pip install evaluate rouge_score bert_score

In [2]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

In [3]:
import torch
import random
import numpy as np
from unsloth import FastLanguageModel
import evaluate
from tqdm import tqdm
import pandas as pd
import re, string
from difflib import SequenceMatcher

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

# Enable TF32 on A100 (big speed-up, minimal quality impact)
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


In [7]:
max_seq_length = 2048
dtype = None
load_in_16bit = True     # fp16 is fine for A100

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_16bit=load_in_16bit,
)
FastLanguageModel.for_inference(model)


==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.495 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,)

In [ ]:
# Test a simple prompt
prompt = "What is Pulmonary fibrosis?"

inputs = tokenizer.apply_chat_template(
    [{"role": "user", "content": prompt}],
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

output_ids = model.generate(
    input_ids=inputs,
    max_new_tokens=200,
    do_sample=False
)

print(tokenizer.decode(output_ids[0], skip_special_tokens=True))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


system

Cutting Knowledge Date: December 2023
Today Date: 08 Dec 2025

user

What is Pulmonary fibrosis?assistant

Pulmonary fibrosis (PF) is a chronic and progressive lung disease characterized by scarring and thickening of the lung tissue, leading to impaired lung function and breathing difficulties. The scarring can be caused by various factors, including:

1. **Idiopathic pulmonary fibrosis (IPF)**: A condition where the exact cause is unknown, but it is believed to be related to genetic and environmental factors.
2. **Exposure to certain substances**: Such as asbestos, silica, and certain chemicals, which can cause inflammation and scarring in the lungs.
3. **Infections**: Such as pneumonia, tuberculosis, and histoplasmosis, which can cause lung damage and scarring.
4. **Autoimmune disorders**: Such as rheumatoid arthritis and lupus, which can cause inflammation and scarring in the lungs.
5. **Genetic disorders**: Such as alpha-1 antitrypsin deficiency, which can cause lung damage

In [ ]:
dataset_path = "mixed_test_1000_no_resources.csv"   
dataset = pd.read_csv(dataset_path)

dataset = dataset.rename(columns={"instruction": "question", "output": "answer"})
print(f"Total samples loaded: {len(dataset)}")

num_eval = min(1000, len(dataset))
print(f"Evaluating {num_eval} samples...\n")

Total samples loaded: 996
Evaluating 996 samples...



In [ ]:

def clean_text(text):
    if not isinstance(text, str):
        return ""
    return (text.replace("Assistant:", "").replace("\n", " ").strip())

def clean_simple(text):
    if not isinstance(text, str): 
        return ""
    text = text.lower()
    text = text.replace("assistant:", "").replace("assistant", "")
    text = re.sub(r"\s+", " ", text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    return text.strip()

def overlap_ratio(pred, ref):
    pw, rw = set(pred.split()), set(ref.split())
    return len(pw & rw) / len(pw) if pw else 0

def sim_ratio(pred, ref):
    return SequenceMatcher(None, pred, ref).ratio()


# Batched Generation

def generate_responses_batched(questions, batch_size=16, max_new_tokens=256):
    """
    Generate deterministic answers for a list of questions using batching.
    """
    all_predictions = []

   
    model.eval()

    for start in tqdm(range(0, len(questions), batch_size), desc="Generating"):
        batch_q = questions[start:start + batch_size]
        # Build list of chat conversations
        messages = [[{"role": "user", "content": q}] for q in batch_q]

        # Tokenize with padding for batch
        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_seq_length,
        ).to(device)

        with torch.inference_mode():
            outputs = model.generate(
                input_ids=inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,   # deterministic
                use_cache=True
            )

        # Decode each sample
        for out_ids, q in zip(outputs, batch_q):
            decoded = tokenizer.decode(out_ids, skip_special_tokens=True)
            # Heuristic: split on question to get answer part
            answer = decoded.split(q)[-1].strip()
            all_predictions.append(clean_text(answer))

    return all_predictions


In [ ]:
# Prediction Loop (Batched)

questions = list(dataset["question"].head(num_eval))
references = [clean_text(a) for a in dataset["answer"].head(num_eval)]

predictions = generate_responses_batched(
    questions,
    batch_size=16,        
    max_new_tokens=256    
)

assert len(predictions) == len(references) == num_eval


# Metrics

print("\nCalculating metrics...")

rouge = evaluate.load("rouge")
rouge_results = rouge.compute(predictions=predictions, references=references)

bertscore = evaluate.load("bertscore")

# BERTScore on GPU, batched
bert_results = bertscore.compute(
    predictions=predictions,
    references=references,
    lang="en",
    rescale_with_baseline=True,
    device=device,
    batch_size=32,    
)

bert_f1_mean = float(np.mean(bert_results["f1"]))


print("\n================= Base Model Evaluation =================")
print(f"Test Samples: {num_eval}")
print(f"ROUGE-L Score: {rouge_results['rougeL']:.4f}")
print(f"BERTScore F1: {bert_f1_mean:.4f}")
print("==========================================================\n")


df = pd.DataFrame({
    "Question": questions,
    "Ground_Truth": references,
    "Model_Prediction": predictions
})
df.to_csv("based_model_eval_results.csv", index=False)
print("Saved → based_model_eval_results.csv")


Generating: 100%|██████████| 63/63 [09:25<00:00,  8.97s/it]



Calculating metrics...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



================= Base Model Evaluation =================
Test Samples: 996
ROUGE-L Score: 0.0814
BERTScore F1: 0.0252

Saved → based_model_eval_results.csv


In [12]:
print(len(predictions))


996


In [ ]:
import torch


# Calculate mean F1, recall, and precision
mean_precision = torch.mean(torch.tensor(bert_results['precision'])).item()
mean_recall = torch.mean(torch.tensor(bert_results['recall'])).item()
mean_f1 = torch.mean(torch.tensor(bert_results['f1'])).item()

print(f"BERTScore Mean Precision: {mean_precision:.4f}")
print(f"BERTScore Mean Recall: {mean_recall:.4f}")
print(f"BERTScore Mean F1: {mean_f1:.4f}")

BERTScore Mean Precision: -0.2271
BERTScore Mean Recall: 0.3041
BERTScore Mean F1: 0.0252


In [ ]:
# Hallucination Detection

hallucinations = []
overlaps, sim_scores = [], []

for p, r in zip(predictions, references):
    cp, cr = clean_simple(p), clean_simple(r)

    ov = overlap_ratio(cp, cr)
    sr = sim_ratio(cp, cr)

    overlaps.append(ov)
    sim_scores.append(sr)

    hallucinations.append(
        len(cp.split()) > 10 and ov < 0.25 and sr < 0.40
    )

df["overlap_ratio"] = overlaps
df["similarity"] = sim_scores
df["hallucination_flag"] = hallucinations

df.to_csv("based_model_eval_with_hallucinations.csv", index=False)
print("Saved → based_model_eval_with_hallucinations.csv")

print("\nHallucination Rate:", df["hallucination_flag"].mean())
print("Done!")

Saved → based_model_eval_with_hallucinations.csv

Hallucination Rate: 0.9979919678714859
Done!
